### 편차보정모델 개발

데이터 로드, 결측제거, 변수 선택

In [18]:
import numpy as np
import pandas as pd

import sys
sys.path.insert(0, '../')
from config.global_params import variable_info


# 인풋 준비
nwp_file = "../DAIO/nwp_data_47105"
obs_file = "../DAIO/obs_data_47105"
nwp_data = np.load(nwp_file)
obs_data = np.load(obs_file)
print(nwp_data.shape)
print(obs_data.shape)


# 결측제거
missing_nwp = set(np.where(np.isnan(nwp_data))[0])
missing_obs = set(np.where(np.isnan(obs_data))[0])
missing_all = list(missing_nwp | missing_obs)
print("결측 합계: ", len(missing_all))

rem_nwp_data = np.delete(nwp_data, missing_all, 0)
rem_obs_data = np.delete(obs_data, missing_all, 0)
print(rem_nwp_data.shape)
print(rem_obs_data.shape)


# 변수선택
sel_var = ['NDNSW_surface', 'UGRD_10m', 'VGRD_10m', 'RH_1_5ma', 'MAXGUST_0m', 'PRMSL_meansealevel']
var_list_dict = list(variable_info.keys())
var_index = [ var_list_dict.index(i) for i in sel_var ]
#print(var_list_dict)
#print(var_index)
sel_rem_nwp_data = rem_nwp_data[:,:,var_index]
rem_obs_data
print(sel_rem_nwp_data.shape)
print(rem_obs_data.shape)

(868, 49, 20)
(868, 49, 2)
결측 합계:  676
(192, 49, 20)
(192, 49, 2)
(192, 49, 6)
(192, 49, 2)


스케일링

In [ ]:
# .. 스케일링 및 데이터 분할
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# .. 데이터 분할
x_tr, x_vl, y_tr, y_vl = train_test_split(x_train_drop, y_train_drop, test_size=0.3, random_state=123, stratify=y_train_drop)

# 스케일링
#scaler = MinMaxScaler()
scaler = StandardScaler()
scaler.fit(x_train_drop)
x_tr_scaled = scaler.transform(x_tr)
x_vl_scaled = scaler.transform(x_vl)
x_test_scaled = scaler.transform(x_test)